In [48]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from  sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import precision_recall_curve, roc_curve, auc, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import KBinsDiscretizer, PolynomialFeatures
from sklearn.impute import SimpleImputer

In [112]:
X = pd.read_csv('../pp_data/train_X.csv')
y = pd.read_csv('../pp_data/train_y.csv')
y = y['ON_TIME_AND_COMPLETE']
test_data = pd.read_csv('../pp_data/public_private_X.csv')

In [113]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

num_features = [
    'DAYS_BETWEEN_ORDER_AND_DUE_DATE',
    'DISTANCE_IN_MILES',
    'GIVEN_TIME_TO_LEAD_TIME_RATIO',
    'LEAD_TIME_TO_DISTANCE_RATIO',
    'AVERAGE_DAILY_DEMAND_CASES',  
    'AVERAGE_VENDOR_ORDER_CYCLE_DAYS',  
    'AVERAGE_ORDER_CYCLE_DAYS'  
]

cat_features = [
    'RESERVABLE_INDICATOR',
    'DIVISION_NUMBER',  
    'DIVISION_CODE', 
    'PRODUCT_CLASSIFICATION',  
    'PRODUCT_MARKET',  
    'PURCHASE_FROM_VENDOR', 
    'SHIP_FROM_VENDOR',  
    'COMPANY_VENDOR_NUMBER',  
    'PURCHASE_ORDER_TYPE'  
]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),  # Impute missing numeric values with the mean
            ('scaler', StandardScaler()),  # Standard scaling
            ('poly', PolynomialFeatures(degree=2, include_bias=False))  # Polynomial terms
        ]), num_features),
        
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing categorical values
            ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore'))  # Handle unseen categories in test data
        ]), cat_features)
    ]
)

pipeline = Pipeline([
    ('preprocessor', preprocessor),  # Transformations
    ('classifier', LogisticRegression())  # Model
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print("Accuracy with default threshold (0.5):", accuracy_score(y_test, y_pred))

y_pred_test_data = pipeline.predict(test_data)

submission = pd.DataFrame({
    'ID': test_data['ID'], 
    'ON_TIME_AND_COMPLETE': y_pred_test_data
})

submission = submission.sort_values(by='ID').reset_index(drop=True)
print("Number of rows in submission:", submission.shape[0])
print("Number of rows in test_data:", test_data.shape[0])
submission.to_csv('prediction.csv', index=False)

/home/baz/ds_folder/stat303_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/baz/ds_folder/stat303_env/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [4, 5, 6, 7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/home/baz/ds_folder/stat303_env/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [3, 4, 5, 6, 7] during transform. These unknown categories will be

Accuracy with default threshold (0.5): 0.772226304188097
Number of rows in submission: 10247
Number of rows in test_data: 10247
